In [ ]:
# STEP 1: CARICAMENTO DOCUMENTI PULITI
import json

from dotenv import load_dotenv
import os

# Specifica il percorso esplicito al file .env
load_dotenv(dotenv_path="config/.env")

openai_api_key = os.getenv("OPENAI_API_KEY")

with open("../parsed/documenti_puliti.json", encoding="utf-8") as f:
    documenti = json.load(f)

# STEP 2: CHUNKING SEMANTICO
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)

chunks = []
metadati = []

for filename, testo in documenti.items():
    sottoparti = text_splitter.split_text(testo)
    for i, chunk in enumerate(sottoparti):
        chunks.append(chunk)
        metadati.append({"source": filename, "chunk_id": i})

print(f"Creati {len(chunks)} chunk totali")

# STEP 3: CREAZIONE EMBEDDINGS (OpenAI o locale)
from langchain.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai_api_key)

# STEP 4: INDICIZZAZIONE CON FAISS
from langchain.vectorstores import FAISS

faiss_index = FAISS.from_texts(chunks, embedding_model, metadatas=metadati)

# STEP 5: SALVATAGGIO INDEX
faiss_index.save_local("../embeddings/faiss_index")
print("Indice FAISS salvato.")


Creati 90 chunk totali


ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model': 'text-embedding...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error